# Human Development Index (HDI) Dataset by the UN
This I think is a pretty full dataset by the UN on Human Development Index. More about it you can find here: https://hdr.undp.org/data-center/human-development-index#/indicies/HDI. \
I have also included in the repository the metadata file (HDR25_Composite_indices_metadata), in which you can find all the indicators and identifiers.\
There are pretty interesting indicators, which for Gender Inequality they have a specific category, with these indicators:\
GII Rank	gii_rank\
Gender Inequality Index (value)	gii\
Maternal Mortality Ratio (deaths per 100,000 live births)	mmr\
Adolescent Birth Rate (births per 1,000 women ages 15-19)	abr\
Population with at least some secondary education, female (% ages 25 and older)	se_f\
Population with at least some secondary education, male (% ages 25 and older)	se_m\
Share of seats in parliament, female (% held by women)	pr_f\
Share of seats in parliament, male (% held by men)	pr_m\
Labour force participation rate, female (% ages 15 and older)	lfpr_f\
Labour force participation rate, male (% ages 15 and older)	lfpr_m\
The letters are an id for the data queries.

In [1]:
import pandas as pd

class HDIData:
    def __init__(self, filepath: str):
        """
        Initialize the HDIData object by loading a CSV file into a DataFrame.
        """
        # Read CSV safely
        self.df = pd.read_csv(filepath, encoding="ISO-8859-1")
        
        # Standardize column names
        self.df.columns = self.df.columns.str.strip().str.lower().str.replace(' ', '_')
        
        # Convert wide-format year columns into long format
        self.long_df = self._reshape_long(self.df)
    
    def _reshape_long(self, df):
        """
        Converts wide-format year columns into long format for easier filtering.
        """
        id_vars = ['iso3', 'country', 'region']  # columns to keep
        value_vars = [col for col in df.columns if any(col.startswith(prefix) for prefix in ['hdi_', 'le_', 'eys_', 'mys_', 'gnipc_', 'gdi_', 'gii_', 'co2_prod_'])]
        
        # Melt the wide columns into long format
        long_df = df.melt(id_vars=id_vars, value_vars=value_vars, var_name='metric_year', value_name='value')
        
        # Split 'metric_year' into 'metric' and 'year'
        long_df[['metric', 'year']] = long_df['metric_year'].str.rsplit('_', n=1, expand=True)
        long_df['year'] = long_df['year'].astype(int)
        long_df.drop(columns='metric_year', inplace=True)
        
        return long_df

    def get_data(self, countries=None, years=None, metric=None):
        """
        Retrieve data for a given metric, list of countries, and/or years.
        """
        df = self.long_df.copy()
        
        # Filter by countries
        if countries is not None:
            if isinstance(countries, str):
                countries = [countries]
            df = df[df['country'].str.lower().isin([c.lower() for c in countries])]
        
        # Filter by years
        if years is not None:
            if isinstance(years, tuple):  # range
                df = df[(df['year'] >= years[0]) & (df['year'] <= years[1])]
            elif isinstance(years, list):
                df = df[df['year'].isin(years)]
            else:  # single year
                df = df[df['year'] == years]
        
        # Filter by metric
        if metric is not None:
            metric = metric.lower().replace(' ', '_')
            df = df[df['metric'] == metric]
        
        return df.reset_index(drop=True)


In [2]:
# Load CSV
hdi = HDIData("HDR25_Composite_indices_complete_time_series.csv")

# Get HDI for Norway, 2010-2020
norway_hdi = hdi.get_data(countries="Norway", years=(2010, 2020), metric="HDI")

# Life expectancy for Japan and Sweden in 2020
life_exp = hdi.get_data(countries=["Nicaragua", "Sweden"], years=(2000, 2020), metric="gii")


In [3]:
print(life_exp)

   iso3    country region  value metric  year
0   NIC  Nicaragua    LAC  0.569    gii  2000
1   SWE     Sweden    NaN  0.059    gii  2000
2   NIC  Nicaragua    LAC  0.573    gii  2001
3   SWE     Sweden    NaN  0.056    gii  2001
4   NIC  Nicaragua    LAC  0.526    gii  2002
5   SWE     Sweden    NaN  0.053    gii  2002
6   NIC  Nicaragua    LAC  0.521    gii  2003
7   SWE     Sweden    NaN  0.048    gii  2003
8   NIC  Nicaragua    LAC  0.524    gii  2004
9   SWE     Sweden    NaN  0.047    gii  2004
10  NIC  Nicaragua    LAC  0.521    gii  2005
11  SWE     Sweden    NaN  0.047    gii  2005
12  NIC  Nicaragua    LAC  0.528    gii  2006
13  SWE     Sweden    NaN  0.046    gii  2006
14  NIC  Nicaragua    LAC  0.515    gii  2007
15  SWE     Sweden    NaN  0.046    gii  2007
16  NIC  Nicaragua    LAC  0.506    gii  2008
17  SWE     Sweden    NaN  0.048    gii  2008
18  NIC  Nicaragua    LAC  0.488    gii  2009
19  SWE     Sweden    NaN  0.049    gii  2009
20  NIC  Nicaragua    LAC  0.478  